In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

import nltk

import string
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('russian')

/home/prop/anaconda3/envs/torch/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
datapath = f'train.csv'
df = pd.read_csv(datapath)
df.head()

,oid,category,text
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...


In [6]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

In [7]:
labels = dict(zip(df.category.unique(), list(range(len(df.category.unique())))))

In [8]:
labels

{'winter_sport': 0,
 'extreme': 1,
 'football': 2,
 'boardgames': 3,
 'hockey': 4,
 'esport': 5,
 'athletics': 6,
 'motosport': 7,
 'basketball': 8,
 'tennis': 9,
 'autosport': 10,
 'martial_arts': 11,
 'volleyball': 12}

# Cleaning text

In [9]:
df[~df.index.isin(df.drop_duplicates(subset=['text']).index)]

,oid,category,text,len
701,763759187,basketball,За кроссовками в Баскетбольный магазин Ghetto ...,11
794,340088263,boardgames,Коллеги традиционно объявляем очередной гильде...,51
852,606047569,basketball,За кроссовками в Баскетбольный магазин Ghetto ...,11
961,681660864,martial_arts,Похудеть возможно даже если. 1. Любишь сладкое...,116
995,846465790,basketball,За кроссовками в Баскетбольный магазин Ghetto ...,11
...,...,...,...,...
38678,424353857,athletics,Одна из самых выгодных карт 33 Обслуживание – ...,47
38700,551674788,motosport,Поздравляем с днем рождения железного человека...,39
38705,508008872,athletics,Вакансия HR менеджер. Обязанности 1. Ознакомле...,202
38717,509545488,esport,Счастливчики которые уносят Кассадина 33 За пр...,43


In [10]:
pd.set_option('display.max_rows', 150)

In [12]:
# check for duplicates using groupby
df_nondupes = df.groupby(['text']).nunique().sort_values(by='category', ascending=False)
# find duplicates with target > 1 as a way of flagging if they are duplicate
df_dupes = df_nondupes[df_nondupes['category'] > 1]
df_dupes.rename(columns={'id':'# of duplicates', 'category':'sum of target var'})
df_dupes.sort_values('text')

,oid,category,len
text,,,
30 дней Okko бесплатно 33 Подключайтесь и смотрите популярные фильмы и сериалы в отличном качестве с доступом на 5 устройств,2,2,1
60 дней Плюса Мульти в подарок по промокоду MYMUSIC 33 Если у вас когда то была подписка Плюс для вас он тоже сработает. Получите миллионы треков аудиокниги и топовые рекомендации которые подойдут именно вам 33,4,3,1
60 дней Плюса Мульти в подарок по промокоду MYMUSIC 33 Если у вас когда то была подписка Плюс для вас он тоже сработает. Получите тысячи треков аудиокниги и топовые рекомендации которые подойдут именно вам 33,7,4,1
Python и Java изучаем языки программирования на практике 12 13 мая в 19. 00 ждем вас на практическом интенсиве по языкам Python и Java За два вечера изучим основы и напишем пару проектов которые сможете показывать работодателям. Научим даже тех кто никогда не прикасался к коду Регистрация по ссылке На интенсиве Расскажем чем занимаются Python разработчики Почему Python подойдет новичкам Как использовать полученные знания Отработаем навыки на тренажере Кто такой Java разработчик и чем он занимается. Какие навыки потребуются для старта в разработке. Как развивается карьера разработчиков почему их зарплаты растут так быстро. Пишем код и пробуем простейшее программирование. Регистрация по ссылке Бонусы для участников. Спикеры Илья Сазонов Руководитель направления tokenoid разработки во Всегда. Да Анастасия Борнева Руководитель направления по исследованию данных в Сбере. Скорее регистрируйтесь и встретимся в прямом эфире,2,2,1
Админ здравствуйте. Пропустите или нет – не знаю так как пост спорный конечно. Мой маленький сын уже больше 2х лет на химиотерапии. Онкология был рецидив. Лечимся в Москве сами из провинции. Никаких сборов не ведем так как лечимся по страховке. Не просим никакой помощи. Но и бесплатное лечение совсем не бесплатное кто лечился – знает. Не говоря об оплате жилья транспортных расходах и сопутствующих. На лечение хватает заработанного самостоятельно но есть проблема в работе. Сына в дет сад не отдашь – опасная среда для его иммунитета внутри имплантирован порт для вливания химии. Работаем из дома – выжигаем картины на дереве. Прошу опубликовать пост – уверена среди ваших подписчиков найдется человек ищущий подарок близким тем более я не беру предоплату. Если не хотите опубликовать бесплатно могу предложить выжечь в качестве оплаты рекламы вам портрет. Документы о лечении ссылку на группу поддержки сына скину в ЛС по вашему запросу. Спасибо.,3,2,1
Ах смотрите какой 33 Хорош правда? Брал у Василия Казакова в восторге от ножа. Соратники мои все обзавидовались побежали заказывать. Качество исполнения поражает все выполнено настолько идеально что придраться не к чему изготовление. Кстати доставили быстро а еще гарантия 5 лет 33 Очень рекомендую tokenoidtokenoid 33,2,2,1
В связи с последними событиями многие россияне не могут платить свои долговые обязательства по кредитам 33 Сейчас есть возможность списать все кредиты кредитные карты микрозаймы долги по ЖКХ на законных основаниях с помощью нового юридического портала Юристы федерального сервиса помогут и все подробно расскажут. Все граждане РФ могут воспользоваться своим правом и обнулить кредиты и долги просто переходите по ссылке ответьте на несколько вопросов и узнайте как это можно сделать. Подробнее по ссылке,7,5,1
В условиях постоянно меняющихся цен часто приходится выбирать что взять сейчас и какая покупка еще потерпит? Чтобы не загонять себя в рамки зарабатывайте больше. Необязательно менять работу просто оформите в пару кликов бесплатную дебетовую Альфа Карту. Платите ей и получайте от 1000 рублей каждый месяц до 2 кэшбэка за все покупки и до 33 у партнеров банка. Чем больше покупок тем больше денег тем больше покупок 33 Оформить,2,2,1
Вам не нужно теперь ломать голову что подарить на праздник. Порадуйте своих близких подарите им бурю эмоций 33 Цены подробности здесь tokentokenoid,8,5,1


In [13]:
df = df[~df.text.isin(df_dupes.reset_index().text)]
df

,oid,category,text,len
0,365271984,winter_sport,Волшебные фото Виктория Поплавская ЕвгенияМедв...,6
1,503385563,extreme,Возвращение в подземелье Треша 33 Эйфория тупо...,43
2,146016084,football,Лучшие чешские вратари – Доминик Доминатор Гаш...,110
3,933865449,boardgames,Rtokenoid Warhammer40k валрак решил нас подкор...,209
4,713550145,hockey,Шестеркин затаскивает Рейнджерс в финал Восточ...,382
...,...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...,61
38736,669736851,autosport,Ира Сидоркова объясняет как сказалась на ее ма...,96
38737,558919241,tennis,24 я ракетка мира хорват Марин Чилич обыграл и...,43
38738,776944963,volleyball,Стал известен календарь мужской сборной России...,143


[nltk_data] Downloading package stopwords to /home/prop/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [16]:
regex = re.compile('[a-zA-Z0-9]')  # компилим regexp выражение

def preprocessing(text):
    text = regex.sub('', text)  # удаляем пунктуацию
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = remove_URL(text)
    text = remove_html(text)
    text = remove_emoji(text)
    text = [token for token in text.split()] # if token not in stop_words
    text = [token.lower() for token  in text]# Удаляем стоп слова
    # text = [stemmer.stem(token) for token in text]  # Выполняем стэмминг
    text = [token for token in text if token]
    text = [token for token in text if not 'tokenoid' in token]
    return ' '.join(text)



df['text'] = df['text'].apply(lambda x: preprocessing(x))

/tmp/ipykernel_6204/2895744398.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: preprocessing(x))


In [18]:
df

,oid,category,text,len
0,365271984,winter_sport,волшебные фото виктория поплавская евгениямедв...,6
1,503385563,extreme,возвращение в подземелье треша 33 эйфория тупо...,43
2,146016084,football,лучшие чешские вратари – доминик доминатор гаш...,110
3,933865449,boardgames,warhammer40k валрак решил нас подкормить не си...,209
4,713550145,hockey,шестеркин затаскивает рейнджерс в финал восточ...,382
...,...,...,...,...
38735,910636962,autosport,8 битная буря снова накрыла пикселями автомоби...,61
38736,669736851,autosport,ира сидоркова объясняет как сказалась на ее ма...,96
38737,558919241,tennis,24 я ракетка мира хорват марин чилич обыграл и...,43
38738,776944963,volleyball,стал известен календарь мужской сборной россии...,143


In [26]:
df_test = pd.read_csv('test.csv')
df_test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


## train

In [17]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [18]:
class TestDataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.oids = df['oid'].tolist()
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]


    def __len__(self):
        return len(self.labels)

    def get_batch_oids(self, idx):
        # Fetch a batch of labels
        return np.array(self.oids[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_oids = self.get_batch_oids(idx)

        return batch_oids, batch_texts


In [19]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
        self.dropout = nn.Dropout(dropout)
        # self.linear_1 = nn.Linear(768, 512)
        # self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(768, 13)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        
        # linear_output1 = self.linear_1(dropout_output)
        # relu_output = self.relu(linear_output1)
        linear_output2 = self.linear_2(dropout_output)
        final_layer = self.sigmoid(linear_output2)

        return final_layer


     

In [20]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=4)

    use_cuda = torch.cuda.is_available()
    # use_cuda=False
    device = torch.device("cuda" if use_cuda else "cpu")
    print('device', device)

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [21]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=4)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [22]:
def test(model, test_data):

    test = TestDataset(test_data)

    # test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()
    
    result = []
    with torch.no_grad():

        for test_oid, test_input in test:

            test_oid = torch.from_numpy(test_oid).to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            pred = output.argmax(dim=1).cpu().detach().numpy()
            max_val = output.max().item()
            result.append([test_oid.item(), pred[0], max_val])
    
    return result

In [23]:
np.random.seed(112)
df_train, df_val = np.split(df.sample(frac=1, random_state=58), 
                                     [int(.95*len(df))])

print(len(df_train),len(df_val))

36338 1913


In [24]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device cuda


100%|██████████| 9085/9085 [31:25<00:00,  4.82it/s]


Epochs: 1 | Train Loss:  0.569 | Train Accuracy:  0.505 | Val Loss:  0.527 | Val Accuracy:  0.663


100%|██████████| 9085/9085 [30:52<00:00,  4.90it/s]


...

Epochs: 8 | Train Loss:  0.435 | Train Accuracy:  0.956 | Val Loss:  0.460 | Val Accuracy:  0.843


100%|██████████| 9085/9085 [30:54<00:00,  4.90it/s]


Epochs: 9 | Train Loss:  0.433 | Train Accuracy:  0.965 | Val Loss:  0.459 | Val Accuracy:  0.843


100%|██████████| 9085/9085 [31:24<00:00,  4.82it/s]


Epochs: 10 | Train Loss:  0.431 | Train Accuracy:  0.970 | Val Loss:  0.459 | Val Accuracy:  0.847


In [27]:
torch.save(model.state_dict(), './save/cleanhalf_2ep.pt')

In [25]:
# model.load_state_dict(torch.load('./save/cleanhalf_2ep.pt'))

<All keys matched successfully>

## Predict

In [25]:
df_test = pd.read_csv('test.csv')
df_test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [26]:
df_test = df_test[~df_test.text.isin(df_dupes.reset_index().text)]
df_test

,oid,text
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...
1,452466036,Естественное восстановление после тяжелой трен...
2,161038103,Тема нарядов продолжается Одна из британских ж...
3,663621910,Привет Избранный. Ты спрашиваешь себя ЧТО здес...
4,566255305,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...
...,...,...
26255,169728316,Выиграй коллекционный пазл по Wortokenoid of W...
26256,279369911,Волейбол от первого лица Егора Пупынина переко...
26257,600699419,Вы были когда нибудь на свидании где вам задав...
26258,560223506,ТОП 20 самых эффективных общефизических упражн...


In [27]:
df_test['text'] = df_test['text'].apply(lambda x: preprocessing(x))

/tmp/ipykernel_6204/1424092570.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['text'] = df_test['text'].apply(lambda x: preprocessing(x))


In [32]:
df_test

,oid,text
0,749208109,спочно сообщество продается за 1300р за покупк...
1,452466036,естественное восстановление после тяжелой трен...
2,161038103,тема нарядов продолжается одна из британских ж...
3,663621910,привет избранный. ты спрашиваешь себя что здес...
4,566255305,король пятисотников в деле андрей рублев успеш...
...,...,...
26255,169728316,выиграй коллекционный пазл по of 33 для участи...
26256,279369911,волейбол от первого лица егора пупынина переко...
26257,600699419,вы были когда нибудь на свидании где вам задав...
26258,560223506,топ 20 самых эффективных общефизических упражн...


In [ ]:
pred = test(model, df_test)

In [29]:
pred = pd.DataFrame(pred, columns=['oid','category', 'prob'])
pred


,oid,category,prob
0,749208109,10,0.946852
1,452466036,7,0.925673
2,161038103,9,0.999033
3,663621910,5,0.995469
4,566255305,9,0.999382
...,...,...,...
26093,169728316,3,0.835856
26094,279369911,12,0.998693
26095,600699419,3,0.999162
26096,560223506,6,0.999019


In [30]:
labels_inv = {v: k for k, v in labels.items()}
labels_inv

{0: 'winter_sport',
 1: 'extreme',
 2: 'football',
 3: 'boardgames',
 4: 'hockey',
 5: 'esport',
 6: 'athletics',
 7: 'motosport',
 8: 'basketball',
 9: 'tennis',
 10: 'autosport',
 11: 'martial_arts',
 12: 'volleyball'}

In [31]:
pred['category'] = pred['category'].map(labels_inv.get)
pred

,oid,category,prob
0,749208109,autosport,0.946852
1,452466036,motosport,0.925673
2,161038103,tennis,0.999033
3,663621910,esport,0.995469
4,566255305,tennis,0.999382
...,...,...,...
26093,169728316,boardgames,0.835856
26094,279369911,volleyball,0.998693
26095,600699419,boardgames,0.999162
26096,560223506,athletics,0.999019


In [32]:
pred.to_csv('./output/bert_clean_output.csv')